# Chapter 8: Markov Random Field

## 0. Introduction

A Bayesian network is a directed, acyclic graph that describes causal relationship between variables. **Markov Random Field (MRF)**, on the other hand, is undirected and non-acyclic. Therefore two main applications of MRF:
(1) modeling memory via the inverse spin glass problem, and (2) inferring the marginal probability (or belief) of a variable in the graph. If some variables are observed, the marginalization will skip those variables. The marginalizatin process is usually intractable as the number in the sum grow exponentially with the number of variables. To reduce the time complexity, **belief propagation** is used to approximate the marginal probability with linear time complexity.

A special case of MRF is the **Conditional Random Field (CRF)**. A CRF consists of the hidden variables and observed variables. The inference problem concerns inferring the distribution of the hidden variables. An example will be OCR: a (true) word forms a chain of hidden variables in the MRF. The hand written characters forms a chain of observed variables. A CRF can be used for error correction in OCR. 

A general rule of thumb is to use Bayesian networks whenever possible, and only switch to MRFs if there is no natural way to model the problem with a directed graph.

[ref](https://ermongroup.github.io/cs228-notes/representation/undirected/)

## 1 Modeling memory: Inverse Ising (spin glass) problem

The inverse Ising problem is used to model patterns in data and hence can be thought of as creating memory for the data.

Recall for an spin glass model with energy function

\begin{equation}
E(s) = \sum_{i,j}J_{ij}s_i s_j - \sum_{i}h_i s_i
\end{equation}

the probability distribution of an Ising model spin configuration $s$ is  

\begin{align}
p(s)&=\exp(-\beta E(s))/Z, \ \ \text{where} \\
Z&=\text{Tr}_{\{s\}}\exp(-\beta E(s))
\end{align}

$Z$ is in general computationally intractable (number of terms $\sim O(2^N)$). Therefore $p(s)$ is difficult in compute. The purpose of variational inference is to approximate $p(s)$ with a simplier (factorized) *variational* distribution $Q(\vec{s})$:  

\begin{align}
Q(\vec{s}) &= \prod_{i=1}^N q_i(s_i) \ \ \text{where} \\
q_i(s_i) &= \frac{\exp(\beta a_i s_i)}{2 \cosh \beta a_i}
\end{align}

The EM algorithm results in the follow pair of iterative equations

\begin{align}
&a^{(t+1)}_i = \sum_j J_{ij} s_j\\
&s^{(t+1)}_i = \tanh(\beta a^{(t+1)}_i)
\end{align}

where $a_i$ can be understood as the mean field at spin $i$.

A spin glass is known to have many local minima. Those minima are basin of attraction of a specific initial condition and can be interpret as memory: given a corrputed input (initial state), under the EM iterative process the state will converge to the basin of attraction corresponds to the initial state. Therefore, to train the spin glass model to memorize patterns in the data (e.g. hand-written digits), one runs an 'inverse Ising problem' to estimate $J_{ij}$ and $h_i$ that best describes the given data $\{s_i\}_{i=1}^N$.

One can factorize $J_{ij}$ using SVD

\begin{equation}
E_\text{Hopfield}(s) = \sum_{i,j}\sum_\mu W_{i\mu}W_{j\mu}s_i s_j - \sum_{i}h_i s_i \label{E_hopfield}\tag{Eq. 1.1}
\end{equation}

The model with energy function of the form above is also known as a **Hopfield model**. The reason to factorize the coupling term $J_{ij}$ is to cast the energy function in a form such that one can express the energy function in terms of visable variables $s$ and latent variables $h$ through the *Hubbard-Stratonovich transformation*. As discussed in previous section, the reason of introducing latent variables is to model the complicated interactions among the visable variables. Expressing the Hopfield model in terms of visable and latent variables while assuming no direct interactions among the hidden variables and the visable variables results in the **Restricted Boltzmann Machine**, which have the following energy function

\begin{equation}
E_{\text{RBM}}(s, h) = \sum_{i,\mu}W_{i\mu}s_i h_\mu - \sum_{i}a_i s_i - \sum_{\mu}b_\mu h_\mu
\end{equation}

the lack of second order terms in $h$ and $v$ indicates no interactions in the latent and visiable layer respectively (hence it is restricted).

### Associative memory

The basin of attractions of the energy function represent the patterns memorized by the Hopfield model. The attractors can be represented by $\{\vec{\xi}^\mu\}_{\mu=1}^P$ where $\vec{\xi}^\mu$ represents the spin configuration corresponds to memory $\mu$. Assume each configuration consists of $N$ spins. The coupling $J_{ij}$ in the Hopfield energy \ref{E_hopfield} can be written as the following to encode the $P$ attractors

\begin{equation}
J_{ij} = \frac{1}{N}\sum_{\mu=1}^P \xi_i^\mu \xi_j^\mu
\end{equation}

The Expectation updating equation becomes

\begin{equation}
a_i = \frac{1}{N}\sum_j \sum_{\mu=1}^P \xi_i^\mu \xi_j^\mu s_j
\end{equation}

Using the above update equation, it can be shown that for any attractor $\vec{\xi}^\alpha$ is indeed a fixed point given
- correlation between configuration $\mu$ and $\alpha$ is statistically zero, i.e. $\rho(\vec{\xi}^\alpha,\vec{\xi}^\mu) \sim P/N$
- $P \ll N$

The first assumption is a strong one and can lead to imperfect memory retrieval in practice.

## 2. Training an RBM

An RBM is trained by maximizing the likelihood of observing the visable variables $s$. 

\begin{align}
\mathcal{L}(W) &= \sum_{m=1}^M \ln P(\vec{s}^{(m)} \mid \{w_{kl}\}) \\
&= \sum_{m=1}^M \text{Tr}_h \ln P(\vec{s}^{(m)}, \vec{h} \mid \{w_{kl}\}) \\
&= \sum_{m=1}^M \text{Tr}_h \exp(-E(\vec{s}^{(m)}, \vec{h}))) - M\ln Z(\{w_{kl}\})
\end{align}

where 

\begin{equation}
P(\vec{s},\vec{h} \mid W) = \frac{e^{-E(\vec{s},\vec{h})}}{Z(W)}
\end{equation}

Applying gradient descent

\begin{align}
\frac{\partial \mathcal{L}(W)}{\partial w_{kl}} &= \sum_{m=1}^M\frac{\text{Tr}_h \exp(-E(\vec{s}^{(m)}, \vec{h}))\ s^{(m)}_k h_l}{\text{Tr}_h \exp(-E(\vec{s}^{(m)}, \vec{h}))} - M \frac{\partial}{\partial w_{kl}}\ln Z(\{w_{kl}\})\\
& \\
&= \langle s_k h_l\rangle_\text{data} - \langle s_k h_l\rangle_\text{model}
\end{align}

$\langle s_k h_l\rangle_\text{data}$ is calculated by fixing $s_k$ from the data and sample $h_l$ from the model. Computing $\langle s_k h_l\rangle_\text{model}$ is very difficult as it involves the joint distribution of $s$ and $h$:

\begin{equation}
\langle s_k h_l\rangle_\text{model} = \text{Tr}_{s_k,h_l}s_k h_l P(s_k, h_l \mid w_{kl})
\end{equation}

One way to compute the term is to use alternating Gibbs sampling.

### Alternating Gibbs sampling

Due to the restricted part of the RBM, the following conditional probabilities can be factorized

\begin{align}
P(\vec{s} \mid \vec{h}) &= \prod_{i=1}^{N_s}P(s_i\mid \vec{h}) \label{RBM_Psh}\tag{Eq 2.1.1}\\ 
P(\vec{h} \mid \vec{s}) &= \prod_{i=1}^{N_h}P(h_i\mid \vec{s}) \label{RBM_Phs}\tag{Eq 2.1.2}\\
\end{align}

Alternating Gibbs sampling is used to sample $s$ and $h$ (while fixing $W, a, b$) through the following alternative process:

\begin{equation}
s^{(0)} \to h^{(0)} \to s^{(1)} \to h^{(1)} \to ...
\end{equation}

Where sample $s^{(t)}$ is drawn from \ref{RBM_Psh} and $h^{(t)}$ is drawn from \ref{RBM_Phs} with $W$ fixed. Faster alternative approach to Gibbs sampling includes *Contrastive Divergence-n* (CD-n), aka blocked truncated Gibbs sampling.

## 3. Belief propagation

[ref](https://www.merl.com/publications/docs/TR2001-22.pdf)

The purpose of beliefe propagation (BP) is to compute the marginal probabilities of all variables in the graph given the evidence (observed value) at a given node.

### 3.1 MFT revisit

Under the MFT ansatz, i.e. 

\begin{align}
p(s) &\approx \prod_i q_i(s_i) \\
q_i(s_i) &= \frac{\exp(\beta a_i s_i)}{2 \cosh \beta a_i}
\end{align}

The average energy of the MF free energy is

\begin{align}
\langle E(s) \rangle_q = \text{Tr}_{\{s\}}\Big[-\sum_{ij}J_{ij} s_i s_j \frac{\exp(\beta a_i s_i)}{2 \cosh \beta a_i}\Big]
\end{align}

### 3.2 Bethe approximation

Instead of factorizing $p(s)$ as the products of marginal probabilities as in MFT, Bethe approximation expresses $p(s)$ in the following way 

\begin{align}
p(s) \approx q(s) = \frac{\prod_{ij}q_{ij}(s_i, s_j)}{\prod_i q_i^{n_i-1}(s_i)}
\end{align}

where $n_i$ is the number of nodes neighboring node $i$. This can be considered as a 'step-up' from MFT, which only consider products of $q_i(s_i)$, and therefore ignore interactions. Expressing the distribution as a products of two-nodes distribution is made possible because of the pairwise interaction nature of the MRF/ Ising model.

The average energy of the Bethe free energy is

\begin{align}
U_{\text{Bethe}}=\langle E(s) \rangle_q &= \text{Tr}_{\{s\}}\Big[-\sum_{ij}J_{ij} s_i s_j \frac{\prod_{ij}q_{ij}(s_i, s_j)}{\prod_i q_i^{n_i-1}(s_i)}\Big] \\
&= -\text{Tr}_{\{s\}}\Big[\sum_{ij}q_{ij}(s_i,s_j) J_{ij} s_i s_j + \sum_i (n_i - 1)\sum_i q_i(s_i) h_i s_i\Big]
\end{align}

and the Bethe entropy is

\begin{align}
S_{\text{Bethe}} &= -\text{Tr}_{\{s\}}\Big[\sum_{ij} q_{ij}(s_i,s_j) \ln q_{ij}(s_i,s_j) - \sum_i (n_i-1)q_i(s_i) \ln q_i(s_i)\Big]\\
\end{align}

together forming the Bethe free energy

\begin{align}
G_\text{Bethe} = U_{\text{Bethe}} - S_{\text{Bethe}}
\end{align}

### 3.3 Equivalence of belief propagation and to the Bethe approximation

The Bethe free energy is identical to the Gibbs free energy of an pairwise MRF with no loops. Similarity, BP is exact on MRF with no loops. Therefore, the BP beliefs are the the minima of the Bethe free energy. It is usually observed that Bethe approximation yield a good results even in MRF with loops.

Just as the EM equations will lead to a stationary solution that corresponds to the minima of the mean-field Ising free energy, the BP equations will lead to a stationiary solution that corresponds to the minima of the Bethe MRF free energy. [ref](https://www.cs.jhu.edu/~ayuille/Pubs10_12/finalchapterAB.pdf)

To illustrate this, construct the following objective to be minimized

\begin{align}
L = G_\text{Bethe} + \sum_{ij} \lambda_{ij}(s_i)\Big(\sum_{j}q_{ij}(s_i,s_j) - q_i(s_i)\Big) + \gamma_{ij}\Big(\sum_{s_i, s_j}q_{ij}(s_i, s_j) - 1\Big) + \gamma_i\Big(\sum_{s_i}q_{i}(s_i) - 1\Big)
\end{align}

where $\lambda_{ij}$, $\gamma_{ij}$ and $\gamma_i$ are the Lagrange multipliers for the marginalization and normalization constraints on the variational marginal and pairwise distribution. Taking derivative of $L$ with respect to $q_i$ and $q_{ij}$ yields equations that describe the minima solutions to $L$.

\begin{align}
\frac{\partial L}{\partial q_{ij}} &= 0 \tag{Eq 3.3.1}\label{L_bethe_1} \\
\frac{\partial L}{\partial q_{i}} &= 0 \tag{Eq 3.3.2}\label{L_bethe_2}
\end{align}

Defining 

\begin{align}
\lambda_{ij}(s_i) \equiv \prod_{k \in N(i) \backslash j} m_{kj}(s_j)
\end{align}

it can be shown that the below equations which constitute the BP algorithm

\begin{align}
m_{ij}(s_j) &\leftarrow \sum_{s_i}\exp(h_i s_i + J_{ij}s_i s_j) \prod_{k \in N(i) \backslash j}m_{ki}(s_i) \\
q_i(s_i) &= k \exp(h_i s_i) \prod_{j \in N(i)}m_{ji}(s_i)
\end{align}

will lead to the fixed points described by \ref{L_bethe_1} and \ref{L_bethe_2}.

## 4 Bayesian network overview
[ref](https://mlg.eng.cam.ac.uk/zoubin/papers/ijprai.pdf)

A Bayesian network is a DAG which specify the conditional dependences among different random variables.

A HMM is a Bayesian network in that
- there are observables $x$ and hidden variable $z$
- the value of the hidden variables $z$ are discrete
- $z$ satisfy the Markov property: $z_t$ only depends on $z_{t-1}$. Similarly, the observable $x_t$ only depends on $z_t$
- $z$ is also allowed to depend on some observable input $u$

Also, an HMM is a Bayesian network with the sense of time: an event can only cause another event in the future, not vice-versa.

Since the hidden variables only take on discrete value, the conditional probabilities between $z_{t-1}$ and $z_t$ are described by the transition matrix.

A state-space model (SSM) is similar to a HMM, except the hidden state can take on continuous values. Because of that, the transition matrix is replaced by some regression function $z_t = f(z_{t-1}) + \epsilon_t$.

Both HMM and SSM are closed under the transition matrix and the regression function respectively, i.e. the HMM hidden state, which follows a multinomial distribution, will remain a multinomial distribution after applying the transition matrix. Similarly, the Linear Gaussian SSM (aka Kalman filter), which follows a Gaussian distribution, will remain the same class of distribution after applying the linear operation and Gaussian noise. This closure property makes inference and learning simple.

### Inference

In the case without hidden variables (i.e. complete data), maximum likelihood (ML) estimation is simple as the log likelihood of the observables can be factorized into products of conditional probabilities given by the Bayesian network. However, ML estimation is difficult with hidden variables as the log likelihood involves marginalizing over the hidden variables, hence involves log of a sum. In this case, ML estimation can be achieved by using EM algorithm.

The EM algorithm in this case does not involve computing the conditional probability $P(z\mid x, \theta)$ but only involve sampling from it. This can be achieved by MCMC methods such as Gibbs sampling.